In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00


In [20]:
data_df

,patientID,trial_num,walking_speed,timeStamps,accel_energy_1,accel_energy_2,accel_energy_3,accel_energy_4,accel_energy_5,accel_energy_6,...,KNEE_RIGHT_Z,ANKLE_RIGHT_Z,FOOT_RIGHT_Z,HEAD_Z,NOSE_Z,EYE_LEFT_Z,EAR_LEFT_Z,EYE_RIGHT_Z,EAR_RIGHT_Z,fraility_class
0,001-LO,Trial1,Regular,0.014806,2.019993e-05,8.896714e-06,1.985365e-07,4.808899e-08,3.431065e-06,8.920357e-06,...,929.656427,825.884006,894.117256,925.016797,954.213183,938.401915,886.661529,968.354628,981.439603,0
1,001-LO,Trial1,Regular,0.034787,4.216104e-06,9.454107e-06,5.867013e-07,1.196821e-06,8.141100e-06,1.967057e-06,...,924.690502,784.109292,847.342565,961.514530,951.963861,963.009239,945.312977,991.238931,1044.996519,1
2,001-LO,Trial1,Regular,0.054767,7.426360e-07,9.197260e-07,6.352156e-07,1.150434e-06,6.814340e-07,5.553919e-07,...,936.771136,777.246423,837.827719,987.262010,983.921111,995.232664,973.751899,1023.415136,1073.302525,0
3,001-LO,Trial1,Regular,0.074748,4.502040e-07,1.693938e-07,9.162256e-08,3.878765e-07,2.871766e-07,4.022295e-07,...,967.044120,802.468369,862.476378,1015.462905,1023.426646,1033.121369,1005.673693,1059.598333,1098.709654,0
4,001-LO,Trial1,Regular,0.094728,2.205772e-07,1.604461e-07,4.103939e-07,2.476341e-07,6.483686e-08,1.801898e-07,...,1013.036813,855.414270,916.645154,1045.783754,1069.454529,1075.905348,1040.604981,1099.243905,1121.611912,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,004-MF,Trial7,Fast,5.108047,1.532015e-08,3.990455e-08,1.139528e-07,6.825939e-08,3.982922e-08,2.491517e-08,...,8914.339654,8913.468519,9095.976202,8465.718946,8528.543198,8496.171499,8470.223444,8471.864473,8375.000232,1
5735,004-MF,Trial7,Fast,5.128027,1.106843e-07,3.870108e-07,2.716906e-05,1.655017e-05,1.203042e-06,1.758924e-08,...,8941.830115,8933.735140,9115.050927,8496.824134,8528.697644,8497.554195,8490.049905,8474.716727,8400.633678,0
5736,004-MF,Trial7,Fast,5.148008,1.383919e-06,1.503486e-06,9.149582e-06,4.712400e-06,3.932854e-06,3.157799e-07,...,8969.320576,8954.001761,9134.125652,8527.929321,8528.852091,8498.936892,8509.876367,8477.568982,8426.267124,1
5737,004-MF,Trial7,Fast,5.167988,9.612178e-07,9.878828e-07,1.545206e-06,1.223737e-06,1.591346e-06,2.058215e-07,...,8996.811038,8974.268382,9153.200377,8559.034508,8529.006537,8500.319589,8529.702828,8480.421236,8451.900570,0


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader, Dataset, Data
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Load your dataset
data_df = pd.read_csv('/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample/KINZ_JOINT_with_acc_temp.csv')

# Split the data into training and testing sets
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Define edges and joints based on your skeleton structure
edges = [
    ('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
    ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
    ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
    ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
    ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
    ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
    ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
    ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
    ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
    ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')
]
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'HEAD', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT', 'CLAVICLE_RIGHT',
    'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT', 'HANDTIP_RIGHT', 'THUMB_RIGHT',
    'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT', 'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'NOSE', 'EYE_LEFT', 'EYE_RIGHT', 'EAR_LEFT', 'EAR_RIGHT'
]
edge_index = torch.tensor([(joints.index(a), joints.index(b)) for a, b in edges], dtype=torch.long).t().contiguous()

# Define the HybridSTGCN Model
class HybridSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, dropout_rate=0.0):
        super(HybridSTGCN, self).__init__()
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)
        self.temporal_conv1 = nn.Conv1d(in_channels=hidden_channels, out_channels=hidden_channels, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))
        batch_size = batch.max().item() + 1
        num_nodes = x.size(0) // batch_size
        x = x.view(batch_size, num_nodes, -1)
        x = x.permute(0, 2, 1)
        x = F.relu(self.temporal_conv1(x))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x.mean(dim=1)
        x = self.dropout(x)
        return x

# Define the Parallel LSTM Model for accel_energy_total and timeStamps
class ParallelLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ParallelLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x.mean(dim=1)
        x = self.fc(x)
        return x

# Define the Combined Model
class CombinedModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, dropout_rate=0.0):
        super(CombinedModel, self).__init__()
        self.skeleton_model = HybridSTGCN(in_channels, hidden_channels, hidden_channels, dropout_rate)
        self.parallel_lstm = ParallelLSTM(input_size=2, hidden_size=hidden_channels)
        self.fc = nn.Linear(hidden_channels * 2, num_classes)

    def forward(self, data, accel_energy_total, timeStamps):
        skeleton_output = self.skeleton_model(data)
        if accel_energy_total.dim() == 1:
            accel_energy_total = accel_energy_total.unsqueeze(1)
        if timeStamps.dim() == 1:
            timeStamps = timeStamps.unsqueeze(1)
        lstm_input = torch.stack([accel_energy_total, timeStamps], dim=2)
        lstm_output = self.parallel_lstm(lstm_input)
        combined_output = torch.cat([skeleton_output, lstm_output], dim=1)
        output = self.fc(combined_output)
        output = torch.sigmoid(output)
        return output.squeeze()

# Custom dataset
class SkeletonDataset(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        row = self.data_df.iloc[idx]
        node_features = []
        for joint in joints:
            x = row[f'{joint}_X']
            y = row[f'{joint}_Y']
            z = row[f'{joint}_Z']
            timestamp = row['timeStamps']
            node_features.append([x, y, z, timestamp])
        x = torch.tensor(node_features, dtype=torch.float)
        accel_energy_total = torch.tensor(row['accel_energy_total'], dtype=torch.float)
        timeStamps = torch.tensor(row['timeStamps'], dtype=torch.float)
        y = torch.tensor(row['fraility_class'], dtype=torch.float)
        data = Data(x=x, edge_index=edge_index, y=y)
        return data, accel_energy_total, timeStamps

# Define the function to train the model and calculate losses and accuracies
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data, accel_energy_total, timeStamps in train_loader:
        data, accel_energy_total, timeStamps = data.to(device), accel_energy_total.to(device), timeStamps.to(device)
        optimizer.zero_grad()
        out = model(data, accel_energy_total, timeStamps)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = (out >= 0.5).float()
        correct += (preds == data.y).sum().item()
        total += data.y.size(0)
    return total_loss / len(train_loader), correct / total

# Define the function to evaluate the model and calculate losses and accuracies
def evaluate_model(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, accel_energy_total, timeStamps in loader:
            data, accel_energy_total, timeStamps = data.to(device), accel_energy_total.to(device), timeStamps.to(device)
            out = model(data, accel_energy_total, timeStamps)
            loss = criterion(out, data.y)
            total_loss += loss.item()
            preds = (out >= 0.5).float()
            correct += (preds == data.y).sum().item()
            total += data.y.size(0)
    return total_loss / len(loader), correct / total

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Prepare the datasets
train_dataset = SkeletonDataset(train_df)
test_dataset = SkeletonDataset(test_df)

# Initialize the model
in_channels = 4
hidden_channels = 64
num_classes = 1
dropout_rate = 0.5

model = CombinedModel(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    num_classes=num_classes,
    dropout_rate=dropout_rate
).to(device)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Create DataLoader for train and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(model, train_loader, optimizer, criterion)
    test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
    print(f"Epoch {epoch + 1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'combined_model.pth')


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1/10, Train Loss: 0.6967, Train Accuracy: 0.5036, Test Loss: 0.6942, Test Accuracy: 0.5035
Epoch 2/10, Train Loss: 0.6954, Train Accuracy: 0.4999, Test Loss: 0.6934, Test Accuracy: 0.5070
Epoch 3/10, Train Loss: 0.6944, Train Accuracy: 0.5151, Test Loss: 0.6945, Test Accuracy: 0.5017
Epoch 4/10, Train Loss: 0.6938, Train Accuracy: 0.5036, Test Loss: 0.6966, Test Accuracy: 0.4678
Epoch 5/10, Train Loss: 0.6934, Train Accuracy: 0.5023, Test Loss: 0.6932, Test Accuracy: 0.5122
Epoch 6/10, Train Loss: 0.6936, Train Accuracy: 0.5056, Test Loss: 0.6923, Test Accuracy: 0.5200
Epoch 7/10, Train Loss: 0.6929, Train Accuracy: 0.5130, Test Loss: 0.6936, Test Accuracy: 0.5157
Epoch 8/10, Train Loss: 0.6924, Train Accuracy: 0.5186, Test Loss: 0.6943, Test Accuracy: 0.5061
Epoch 9/10, Train Loss: 0.6934, Train Accuracy: 0.5084, Test Loss: 0.6970, Test Accuracy: 0.4730
Epoch 10/10, Train Loss: 0.6930, Train Accuracy: 0.5164, Test Loss: 0.6973, Test Accuracy: 0.4678
